In [ ]:
!pip install --upgrade sagemaker==2.117.0

# PyTorch NLP

In [ ]:
import boto3
import json
import sagemaker

from pytorch_nlp.pipeline import PyTorchPipeline
import sagemaker

role = sagemaker.get_execution_role()

sm_pipeline = PyTorchPipeline(
    role=role,
    lambda_role="arn:aws:iam::<>:role/deployment-role",
    region="",
    default_bucket="",
    project_name="pytorch-nlp",
).construct_pipeline()

In [ ]:
# don't run this due to eval bug with PyTorchProcessor

# definition = json.loads(sm_pipeline.definition())
# definition

In [ ]:
upsert_response = sm_pipeline.upsert(role_arn=role)

In [ ]:
print("\n###### Created/Updated SageMaker Pipeline: Response received:")
print(upsert_response)

execution = sm_pipeline.start(
    parameters=dict(
        ModelApprovalStatus="Approved",
        InferenceMethod="serverless",  # or "realtime"
        TrainData="s3://<>/tweet-sentiment-extraction/train.csv",
    ),
)
print(f"\n###### Execution started with PipelineExecutionArn: {execution.arn}")

print("Waiting for the execution to finish...")
execution.wait()
print("\n#####Execution completed. Execution step details:")

execution.list_steps()

# Invoke endpoint

In [ ]:
import json

runtime = boto3.client("sagemaker-runtime")

endpoint_name = ""

In [ ]:
content_type = "application/json"
payload = {"text": "I feel so happy right now", "sentiment": "positive"}
# payload = {"text": "Last session of the day http://twitpic.com/67ezh", "sentiment": "neutral"}

response = runtime.invoke_endpoint(
    EndpointName=endpoint_name, ContentType=content_type, Body=json.dumps(payload)
)

print(json.loads(response["Body"].read()))

# Teardown endpoint

In [ ]:
sm_client = sagemaker.Session().sagemaker_client

endpoint_name = ""
endpoint_config_name = ""

In [ ]:
# Delete the EndpointConfig
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)

# Delete the Endpoint
sm_client.delete_endpoint(EndpointName=endpoint_name)